In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.3 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/graph

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [9]:
import pandas as pd
d = {'x': [7, 17, 27, 37, 47, 57], 'y': [7, 18, 16, 24, 22, 30]}
df = pd.DataFrame(data=d)
df1=spark.createDataFrame(df)
df1.show()

+---+---+
|  x|  y|
+---+---+
|  7|  7|
| 17| 18|
| 27| 16|
| 37| 24|
| 47| 22|
| 57| 30|
+---+---+



In [11]:
from pyspark.ml.feature import VectorAssembler
assembleur = VectorAssembler(inputCols=df1.columns, outputCol="caractéristiques")
df_nouv=assembleur.transform(df1)
df_nouv.show()

+---+---+----------------+
|  x|  y|caractéristiques|
+---+---+----------------+
|  7|  7|       [7.0,7.0]|
| 17| 18|     [17.0,18.0]|
| 27| 16|     [27.0,16.0]|
| 37| 24|     [37.0,24.0]|
| 47| 22|     [47.0,22.0]|
| 57| 30|     [57.0,30.0]|
+---+---+----------------+



In [13]:
from pyspark.ml.stat import Correlation
pearson_corr = Correlation.corr(df_nouv,'caractéristiques')
pearson_corr.show(2,False)

+---------------------------------------------------------------------------------+
|pearson(caractéristiques)                                                        |
+---------------------------------------------------------------------------------+
|1.0                 0.9201575383978458  
0.9201575383978458  1.0                 |
+---------------------------------------------------------------------------------+



In [14]:
spearman_corr=Correlation.corr(df_nouv,'caractéristiques',"spearman")
spearman_corr.show(2,False)

+---------------------------------------------------------------------------------+
|spearman(caractéristiques)                                                       |
+---------------------------------------------------------------------------------+
|1.0                 0.8857142857142843  
0.8857142857142843  1.0                 |
+---------------------------------------------------------------------------------+

